# Problema
El objetivo es desarrollar una aplicación que genere automáticamente un folleto resumido en formato Markdown a partir del contenido de la página web de una empresa. Para lograr esto, se empleará un modelo de lenguaje de gran escala (LLM) que, mediante técnicas de web scraping, extraerá el texto relevante y los enlaces de la página principal. Posteriormente, el LLM analizará estos enlaces para identificar aquellos que aporten información valiosa sobre la empresa. El contenido de los enlaces seleccionados se recopilará y consolidará con el de la página principal. Finalmente, el LLM generará un folleto en formato Markdown que resuma la información más destacada de la empresa, proporcionando una visión clara y concisa de sus servicios, productos y valores.

![SNOWFALL](app_folleto_diagram.png) 

In [79]:
import openai
import gradio as gr
import requests
from bs4 import BeautifulSoup
import os
from dotenv import load_dotenv
import json

In [80]:
load_dotenv()
openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key existe y empieza por {openai_api_key[:8]}")

OpenAI API Key existe y empieza por sk-proj-


In [81]:
MODEL = 'gpt-4o-mini'

In [82]:
def agente_scraping(url):
    """
    Extrae el contenido textual y los enlaces de una página web dada.
    """
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')

        # Elimina elementos no deseados
        for tag in soup(['script', 'style', 'img', 'input']):
            tag.decompose()

        # Extrae el texto principal
        texto_principal = soup.get_text(separator='\n', strip=True)

        # Extrae todos los enlaces de la página
        enlaces = [a['href'] for a in soup.find_all('a', href=True)]

        return texto_principal, enlaces
    except Exception as e:
        return str(e), []

In [83]:
def agente_analisis_enlaces(texto_principal, enlaces):
    link_system_prompt = "Eres un asistente que ayuda a identificar enlaces relevantes en el contenido de una página web."
    user_prompt = (
        f"Contenido de la página:\n\n{texto_principal}\n\n"
        f"Enlaces extraídos:\n{enlaces}\n\n"
        "Identifica cuáles de estos enlaces son relevantes para incluir en un folleto resumido de la empresa."
    )

    link_system_prompt = "Se te proporciona una lista de enlaces que se encuentran en una página web. \
    Puedes decidir cuáles de los enlaces serían los más relevantes para incluir en un folleto sobre la empresa, \
    como enlaces a una página Acerca de, una página de la empresa, las carreras/empleos disponibles o páginas de Cursos/Packs.\n"
    link_system_prompt += "Debes responder en JSON como en este ejemplo:"
    link_system_prompt += """
    {
        "links": [
            {"type": "Pagina Sobre nosotros", "url": "https://url.completa/aqui/va/sobre/nosotros"},
            {"type": "Pagina de Cursos": "url": "https://otra.url.completa/courses"}
        ]
    }
    """

    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": user_prompt}
        ], 
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [84]:
def agente_generacion_folleto(contenidos):
    brochure_system_prompt = (
        "Eres un asistente experto en marketing que crea folletos atractivos en formato Markdown."
    )
    user_prompt = (
        f"Contenido de la empresa y sus secciones relevantes:\n\n{contenidos}\n\n"
        "Genera un folleto resumido en formato Markdown que destaque la información más importante de la empresa."
    )

    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": brochure_system_prompt},
            {"role": "user", "content": user_prompt}
        ]
    )
    result = response.choices[0].message.content
    return result

In [85]:
def generar_folleto(url):
    """
    Función principal que coordina los agentes y genera el folleto final.
    """
    # Agente de Scraping: obtiene el contenido de la página y los enlaces
    texto_principal, enlaces = agente_scraping(url)
    if not texto_principal:
        return "No se pudo extraer contenido de la página principal."

    # Agente de Análisis de Enlaces: identifica los enlaces relevantes
    enlaces_relevantes = agente_analisis_enlaces(texto_principal, enlaces)
    if not enlaces_relevantes:
        return "No se identificaron enlaces relevantes."

    # Obtener contenido de los enlaces relevantes
    contenidos = texto_principal
    for enlace in enlaces_relevantes:
        if not enlace.startswith('http'):
            enlace = requests.compat.urljoin(url, enlace)  # Normaliza la URL relativa a absoluta
        contenido_enlace, _ = agente_scraping(enlace)
        contenidos += f"\n\n{contenido_enlace}"

    # Agente de Generación de Folleto: genera el folleto en Markdown
    folleto = agente_generacion_folleto(contenidos)
    return folleto

In [ ]:
interface = gr.Interface(
    fn=generar_folleto,
    inputs=gr.Textbox(label="URL de la página web de la empresa"),
    outputs=gr.Markdown(label="Folleto generado"),
    title="Generador de Folletos Empresariales",
    description="Ingresa la URL de la página web de una empresa para generar un folleto resumido en formato Markdown utilizando GPT.",
    flagging_mode="never"
)

interface.launch()